# 1. Data generation for service evaluation
### TP <--> Data --> Metrics --> DSM

__This is a fully contained script that generates a DSM, initializes it throught metrics of the system adn then optimizes it. Eventually, it is a service design optimizer that compares results to baseline. The case is a last mile delivery service and the results are at the end. Optimization of single TP and TP ascociation are done to derive the best DSM and then provided.__



In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

np.random.seed()

# =====================================================
# 1. SERVICE TOUCHPOINTS (Bai nodes)
# =====================================================
TOUCHPOINTS = [
    "GenerateDemand",
    "NavigateRoute",
    "TrafficAssessment",
    "LoadCheck",
    "PackageHandling",
    "DeliveryExecution",
    "PaymentConfirmation",
    "Feedback"
]

# =====================================================
# 2. PRE-OPTIMIZED (AS-IS) PROCESS ORDER
# =====================================================
PRE_OPTIMIZED_ORDER = TOUCHPOINTS.copy()

# =====================================================
# 3. BAI-STYLE DSM (FUZZY EXTENSION)
# =====================================================
DSM = np.array([
    [0,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   0.7, 0,   0,   1,   0,   0],
    [0,   0.6, 0,   0.4, 0,   1,   0,   0],
    [0,   0,   0.5, 0,   0.8, 1,   0,   0],
    [0,   0,   0,   0.6, 0,   1,   0,   0],
    [0,   0,   0.4, 0,   0,   0,   1,   0],
    [0,   0,   0,   0,   0,   0,   0,   1],
    [0,   0,   0,   0,   0,   0,   0,   0],
])

# =====================================================
# 4. BUILD DSM GRAPH
# =====================================================
G = nx.DiGraph()
for i, u in enumerate(TOUCHPOINTS):
    for j, v in enumerate(TOUCHPOINTS):
        if DSM[i, j] > 0:
            G.add_edge(u, v, weight=DSM[i, j])

# =====================================================
# 5. STRONGLY CONNECTED COMPONENTS (COUPLED MODULES)
# =====================================================
SCCS = list(nx.strongly_connected_components(G))

# =====================================================
# 6. REDUCED GRAPH + LEVELING (BAI DEFINITION 5)
# =====================================================
module_map = {}
for idx, comp in enumerate(SCCS):
    for node in comp:
        module_map[node] = idx

MG = nx.DiGraph()
for u, v in G.edges():
    mu, mv = module_map[u], module_map[v]
    if mu != mv:
        MG.add_edge(mu, mv)

LEVELS = list(nx.topological_generations(MG))

# =====================================================
# 7. TOUCHPOINT DATA GENERATORS
# =====================================================
def gen_generate_demand():
    return {"demand_volume": np.random.randint(1, 50), "urgency": np.random.rand()}

def gen_navigate_route():
    return {"distance_km": np.random.uniform(1, 200), "planned_speed": np.random.uniform(30, 80)}

def gen_traffic_assessment():
    return {"congestion": np.random.rand(), "weather_risk": np.random.rand()}

def gen_load_check():
    return {"capacity_ratio": np.random.uniform(0.2, 1.0), "current_load": np.random.randint(1, 200)}

def gen_package_handling():
    return {"handling_time_min": np.random.uniform(5, 60), "fragility": np.random.rand()}

def gen_delivery_execution():
    return {"actual_speed": np.random.uniform(20, 90), "delay_minutes": np.random.uniform(0, 120)}

def gen_payment_confirmation():
    return {"payment_success": np.random.choice([0, 1]), "processing_time_sec": np.random.uniform(5, 60)}

def gen_feedback():
    return {"satisfaction": np.random.uniform(0, 5), "complaint": np.random.choice([0, 1], p=[0.85, 0.15])}

GENERATOR_MAP = {
    "GenerateDemand": gen_generate_demand,
    "NavigateRoute": gen_navigate_route,
    "TrafficAssessment": gen_traffic_assessment,
    "LoadCheck": gen_load_check,
    "PackageHandling": gen_package_handling,
    "DeliveryExecution": gen_delivery_execution,
    "PaymentConfirmation": gen_payment_confirmation,
    "Feedback": gen_feedback,
}

# =====================================================
# 8. EXECUTION FUNCTIONS (LONG-FORM, NO NaNs)
# =====================================================
def run_pre_optimized(order):
    rows = []
    for step, tp in enumerate(order, 1):
        for k, v in GENERATOR_MAP[tp]().items():
            rows.append({
                "process_state": "pre_optimized",
                "touchpoint": tp,
                "step": step,
                "variable": k,
                "value": v
            })
    return pd.DataFrame(rows)

def run_post_optimized(levels, sccs):
    rows = []
    for lvl, modules in enumerate(levels, 1):
        for m in modules:
            for tp in sccs[m]:
                for k, v in GENERATOR_MAP[tp]().items():
                    rows.append({
                        "process_state": "post_optimized",
                        "touchpoint": tp,
                        "level": lvl,
                        "module": m,
                        "variable": k,
                        "value": v
                    })
    return pd.DataFrame(rows)

# =====================================================
# 9. RUN EVERYTHING
# =====================================================
df_pre = run_pre_optimized(PRE_OPTIMIZED_ORDER)
df_post = run_post_optimized(LEVELS, SCCS)

df_all = pd.concat([df_pre, df_post], ignore_index=True)

# =====================================================
# 10. OUTPUT SUMMARY
# =====================================================
print("\n=== PRE-OPTIMIZED SAMPLE ===")
print(df_pre.head())

print("\n=== POST-OPTIMIZED SAMPLE ===")
print(df_post.head())

print("\n=== HIERARCHICAL LEVELS (BAI) ===")
for i, lvl in enumerate(LEVELS, 1):
    print(f"Level {i}: {[list(SCCS[m]) for m in lvl]}")

D_graph = 8
candidate_dims = [[2],[2],[2],[2],[2],[2],[2],[2]]#,[1],]


# 2. Dummy DSM optimization

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ==========================================
# 1. SETUP & DATA GENERATION (From Input)
# ==========================================
np.random.seed(42) # Fixed seed for reproducibility
N = 1000  # Reduced N slightly for speed in demonstration

# --- Domain 1: Route & Navigation ---
route_nav = pd.DataFrame({
    'distance_km': np.random.uniform(1, 200, N),
    'planned_speed_kmh': np.random.uniform(20, 80, N),
    'actual_speed_kmh': np.random.uniform(15, 90, N),
    'eta_minutes': np.random.uniform(10, 300, N),
    'route_risk_score': np.random.uniform(0, 1, N),
    'fuel_capacity_l': np.random.uniform(20, 200, N),
    'fuel_used_l': np.random.uniform(5, 180, N),
})
route_nav['speed_variance'] = route_nav['planned_speed_kmh'] - route_nav['actual_speed_kmh']

# --- Domain 2: Traffic & Weather ---
traffic_weather = pd.DataFrame({
    'traffic_congestion': np.random.uniform(0, 1, N),
    'road_condition_score': np.random.uniform(0, 1, N),
    'visibility_km': np.random.uniform(0.5, 10, N),
    'storm_probability': np.random.uniform(0, 1, N),
    'precipitation_mm': np.random.uniform(0, 50, N),
})

# --- Domain 3: Delivery Load ---
delivery_load = pd.DataFrame({
    'max_packages': np.random.randint(10, 200, N),
    'current_packages': np.random.randint(1, 200, N),
    'package_weight_kg': np.random.uniform(0.5, 50, N),
})
delivery_load['load_ratio'] = delivery_load['current_packages'] / delivery_load['max_packages']

# --- Domain 4: Package Handling ---
package_handling = pd.DataFrame({
    'package_type': np.random.choice(['standard', 'fragile', 'perishable', 'oversize'], N),
    'handling_speed_pph': np.random.uniform(5, 50, N),
    'storage_temp_req_C': np.random.uniform(-5, 25, N),
})
# Numeric encoding for matrix
package_handling['package_type_id'] = pd.factorize(package_handling['package_type'])[0]
package_handling_numeric = package_handling.drop(columns=['package_type'])

# --- Domain 5: Temporal & Hub ---
temporal_hub = pd.DataFrame({
    'pickup_hour': np.random.randint(0, 24, N),
    'pickup_day': np.random.randint(0, 7, N),
    'hub_congestion_level': np.random.uniform(0, 1, N),
    'delivery_delay_minutes': np.random.uniform(0, 120, N),
})

# List of dataframes (Nodes)
domains = [route_nav, traffic_weather, delivery_load, package_handling_numeric, temporal_hub]
domain_names = ["Route", "Traffic/Wx", "Load", "Handling", "Hub/Time"]

# Combine for global matrix
DATA_MATRIX = np.hstack([df.values for df in domains])
# Normalize
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0) + 1e-8)

# ==========================================
# 2. DSM CONSTRUCTION
# ==========================================

def calculate_dsm(domains, domain_names):
    """
    Calculates the Design Structure Matrix (DSM) based on correlation
    between the features of different domains.
    """
    n_domains = len(domains)
    dsm = np.zeros((n_domains, n_domains))

    # 1. Global Correlation Matrix of all raw features
    all_data = pd.concat(domains, axis=1)
    # Ensure all data is numeric
    all_data = all_data.select_dtypes(include=[np.number])
    corr_matrix = all_data.corr().abs()

    # 2. Block-wise aggregation to Node-level DSM
    # We iterate through the columns belonging to domain i and domain j
    # and average their interaction strength.

    col_idx = 0
    domain_ranges = []
    for df in domains:
        cols = df.select_dtypes(include=[np.number]).shape[1]
        domain_ranges.append((col_idx, col_idx + cols))
        col_idx += cols

    for i in range(n_domains):
        for j in range(n_domains):
            if i == j:
                dsm[i, j] = 1.0 # Self-dependency
            else:
                r_i = domain_ranges[i]
                r_j = domain_ranges[j]

                # Extract the sub-block of correlations between domain i and j
                # We use iloc on the correlation matrix
                sub_block = corr_matrix.iloc[r_i[0]:r_i[1], r_j[0]:r_j[1]]

                # The DSM value is the mean interaction strength
                dsm[i, j] = sub_block.values.mean()

    return pd.DataFrame(dsm, index=domain_names, columns=domain_names)

DSM = calculate_dsm(domains, domain_names)

print("--- Design Structure Matrix (Interaction Density) ---")
print(DSM.round(3))
print("\n")

# ==========================================
# 3. METRIC EVALUATION (Cost & Quality)
# ==========================================
import numpy as np
import pandas as pd
import networkx as nx

np.random.seed()

# =====================================================
# 1. SERVICE TOUCHPOINTS (BAI Nodes)
# =====================================================
TOUCHPOINTS = [
    "GenerateDemand",
    "NavigateRoute",
    "TrafficAssessment",
    "LoadCheck",
    "PackageHandling",
    "DeliveryExecution",
    "PaymentConfirmation",
    "Feedback"
]

# =====================================================
# 2. PRE-OPTIMIZED (AS-IS) PROCESS ORDER
# =====================================================
PRE_OPTIMIZED_ORDER = TOUCHPOINTS.copy()

# =====================================================
# 3. BAI-STYLE DSM (FUZZY DEPENDENCIES)
# =====================================================
DSM_MATRIX = np.array([
    [0,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   0.7, 0,   0,   1,   0,   0],
    [0,   0.6, 0,   0.4, 0,   1,   0,   0],
    [0,   0,   0.5, 0,   0.8, 1,   0,   0],
    [0,   0,   0,   0.6, 0,   1,   0,   0],
    [0,   0,   0.4, 0,   0,   0,   1,   0],
    [0,   0,   0,   0,   0,   0,   0,   1],
    [0,   0,   0,   0,   0,   0,   0,   0],
])

DSM = pd.DataFrame(
    DSM_MATRIX,
    index=TOUCHPOINTS,
    columns=TOUCHPOINTS
)

# =====================================================
# 4. BUILD DSM GRAPH
# =====================================================
G = nx.DiGraph()
for i, u in enumerate(TOUCHPOINTS):
    for j, v in enumerate(TOUCHPOINTS):
        if DSM_MATRIX[i, j] > 0:
            G.add_edge(u, v, weight=DSM_MATRIX[i, j])

# =====================================================
# 5. STRONGLY CONNECTED COMPONENTS (COUPLED MODULES)
# =====================================================
SCCS = list(nx.strongly_connected_components(G))

# =====================================================
# 6. REDUCED GRAPH + LEVELING (BAI DEFINITION 5)
# =====================================================
module_map = {}
for idx, comp in enumerate(SCCS):
    for node in comp:
        module_map[node] = idx

MG = nx.DiGraph()
for u, v in G.edges():
    mu, mv = module_map[u], module_map[v]
    if mu != mv:
        MG.add_edge(mu, mv)

LEVELS = list(nx.topological_generations(MG))

# =====================================================
# 7. TOUCHPOINT DATA GENERATORS
# =====================================================
def gen_generate_demand():
    return {"demand_volume": np.random.randint(1, 50), "urgency": np.random.rand()}

def gen_navigate_route():
    return {"distance_km": np.random.uniform(1, 200), "planned_speed": np.random.uniform(30, 80)}

def gen_traffic_assessment():
    return {"congestion": np.random.rand(), "weather_risk": np.random.rand()}

def gen_load_check():
    return {"capacity_ratio": np.random.uniform(0.2, 1.0), "current_load": np.random.randint(1, 200)}

def gen_package_handling():
    return {"handling_time_min": np.random.uniform(5, 60), "fragility": np.random.rand()}

def gen_delivery_execution():
    return {"actual_speed": np.random.uniform(20, 90), "delay_minutes": np.random.uniform(0, 120)}

def gen_payment_confirmation():
    return {"payment_success": np.random.choice([0, 1]), "processing_time_sec": np.random.uniform(5, 60)}

def gen_feedback():
    return {"satisfaction": np.random.uniform(0, 5), "complaint": np.random.choice([0, 1], p=[0.85, 0.15])}

GENERATOR_MAP = {
    "GenerateDemand": gen_generate_demand,
    "NavigateRoute": gen_navigate_route,
    "TrafficAssessment": gen_traffic_assessment,
    "LoadCheck": gen_load_check,
    "PackageHandling": gen_package_handling,
    "DeliveryExecution": gen_delivery_execution,
    "PaymentConfirmation": gen_payment_confirmation,
    "Feedback": gen_feedback,
}

# =====================================================
# 8. EXECUTION FUNCTIONS
# =====================================================
def run_pre_optimized(order):
    rows = []
    for step, tp in enumerate(order, 1):
        for k, v in GENERATOR_MAP[tp]().items():
            rows.append({
                "process_state": "pre_optimized",
                "touchpoint": tp,
                "step": step,
                "variable": k,
                "value": v
            })
    return pd.DataFrame(rows)

def run_post_optimized(levels, sccs):
    rows = []
    for lvl, modules in enumerate(levels, 1):
        for m in modules:
            for tp in sccs[m]:
                for k, v in GENERATOR_MAP[tp]().items():
                    rows.append({
                        "process_state": "post_optimized",
                        "touchpoint": tp,
                        "level": lvl,
                        "module": m,
                        "variable": k,
                        "value": v
                    })
    return pd.DataFrame(rows)

# =====================================================
# 9. RUN
# =====================================================
df_pre = run_pre_optimized(PRE_OPTIMIZED_ORDER)
df_post = run_post_optimized(LEVELS, SCCS)

df_all = pd.concat([df_pre, df_post], ignore_index=True)

class MetricsEvaluator:
    def __init__(self):
        self.optimizer = BranchBoundOptimizer(minimize=False)

        self.metric_feature_map = {
            'DeliveryTime': ['Distance', 'Speed', 'Load'],
            'TravelDistance': ['Distance', 'PackageSize'],
            'OperationalCost': ['Distance', 'Load', 'Capacity'],
            'EnergyUse': ['Distance', 'Speed'],
            'CustomerSatisfaction': ['DeliveryTime']
        }

    def compute_system_metrics(self, df_exec):
        features = extract_features_from_execution(df_exec)

        # --- Operational Cost ---
        val_cost = self.optimizer.optimize(
            features={k: features[k] for k in self.metric_feature_map['OperationalCost']},
            metric_mask=[0, 0, 1, 0, 0]
        )
        operational_cost = 0.2 * val_cost + 0.1

        # --- Quality (Time + Satisfaction) ---
        val_quality = self.optimizer.optimize(
            features=features,
            metric_mask=[1, 0, 0, 0, 1]
        )
        quality = (
            METRIC_FORMULAS[0](val_quality) +
            METRIC_FORMULAS[4](val_quality)
        )

        return operational_cost, quality
METRIC_FORMULAS = [
    lambda x: np.exp(-x / 120),   # DeliveryTime
    lambda x: np.exp(-x / 100),   # TravelDistance
    lambda x: 0.2 * x + 0.1,      # OperationalCost
    lambda x: x / 50,             # EnergyUse
    lambda x: np.exp(-x / 60),    # CustomerSatisfaction
]
class BranchBoundOptimizer:
    def __init__(self, tol=1e-3, max_depth=20, minimize=True, value_range=(0.0, 5.0)):
        self.tol = tol
        self.max_depth = max_depth
        self.minimize = minimize
        self.value_range = value_range

    def optimize(self, features, y=None, metric_mask=None):
        base = np.mean(list(features.values())) if features else 0.5
        a0, b0 = self.value_range
        a0 += base
        b0 += base

        work = [(a0, b0, 0)]
        best_x = None
        best_score = np.inf if self.minimize else -np.inf

        while work:
            a, b, depth = work.pop()
            mid = 0.5 * (a + b)

            values = [
                f(mid) if m else 0.0
                for f, m in zip(METRIC_FORMULAS, metric_mask)
            ]
            score = sum(values)

            if best_x is None or (
                score < best_score if self.minimize else score > best_score
            ):
                best_x = mid
                best_score = score

            if depth < self.max_depth and (b - a) > self.tol:
                work.append((a, mid, depth + 1))
                work.append((mid, b, depth + 1))

        return best_x
def extract_features_from_execution(df):
    features = {}

    # Route & execution
    features["Distance"] = df.query("variable == 'distance_km'")["value"].mean()
    features["Speed"] = df.query("variable == 'actual_speed'")["value"].mean()

    # Load & capacity
    features["Load"] = df.query("variable == 'current_load'")["value"].mean()
    features["Capacity"] = df.query("variable == 'capacity_ratio'")["value"].mean()

    # Package attributes
    features["PackageSize"] = df.query("variable == 'fragility'")["value"].mean()
    features["TempRequirement"] = df.query("variable == 'handling_time_min'")["value"].mean()

    # Delivery time proxy
    features["DeliveryTime"] = df.query("variable == 'delay_minutes'")["value"].mean()

    # Fill missing values safely
    for k, v in features.items():
        if np.isnan(v):
            features[k] = 0.5

    return features
evaluator = MetricsEvaluator()
avg_op_cost, avg_quality = evaluator.compute_system_metrics(df_post)

# Structural cost from BAI DSM (not correlation DSM)
structural_cost = DSM_MATRIX.sum()

print("\n=== CALCULATION RESULTS ===")
print(f"1. Structural Complexity Cost (DSM Coupling): {structural_cost:.4f}")
print(f"2. Operational Cost Index:                  {avg_op_cost:.4f}")
print(f"3. System Quality Score:                    {avg_quality:.4f}")


# 3. Metric evaluator and Optimizer

In [ ]:
import numpy as np

# ======================================================
# METRIC KEYS (Node-level metrics)
# ======================================================
METRIC_KEYS = [
    "GenerateDemand",
    "NavigateRoute",
    "TrafficAssessment",
    "LoadCheck",
    "PackageHandling",
    "DeliveryExecution",
    "PaymentConfirmation",
    "Feedback",
    "Aggregate"
]
FEATURE_KEYS = [
    'demand_volume',
    'urgency',
    'distance_km',
    'planned_speed',
    'congestion',
    'weather_risk',
    'capacity_ratio',
    'current_load',
    'handling_time_min',
    'fragility',
    'actual_speed',
    'delay_minutes',
    'payment_success',
    'processing_time_sec',
    'satisfaction',
    'complaint'
]
# ======================================================
# METRIC TARGET
# Rows = nodes, Columns = metrics
# ======================================================
METRIC_TARGET = [
    # GD NR TA LC PH DE PC FB AG
    [1, 0, 0, 0, 0, 0, 0, 0, 1],  # GenerateDemand
    [0, 1, 0, 0, 0, 0, 0, 0, 1],  # NavigateRoute
    [0, 0, 1, 0, 0, 0, 0, 0, 1],  # TrafficAssessment
    [0, 0, 0, 1, 0, 0, 0, 0, 1],  # LoadCheck
    [0, 0, 0, 0, 1, 0, 0, 0, 1],  # PackageHandling
    [0, 0, 0, 0, 0, 1, 0, 0, 1],  # DeliveryExecution
    [0, 0, 0, 0, 0, 0, 1, 0, 1],  # PaymentConfirmation
    [0, 0, 0, 0, 0, 0, 0, 1, 1],  # Feedback
    [0, 0, 0, 0, 0, 0, 0, 0, 1],  # Aggregate
]

# ======================================================
# METRIC FORMULAS
# ======================================================
METRIC_FORMULAS = [
    lambda x: np.exp(-x),          # GenerateDemand
    lambda x: np.exp(-x),          # NavigateRoute
    lambda x: 0.2 * x + 0.1,       # TrafficAssessment
    lambda x: x,                   # LoadCheck
    lambda x: x,                   # PackageHandling
    lambda x: np.exp(-x),          # DeliveryExecution
    lambda x: np.exp(-x),          # PaymentConfirmation
    lambda x: x,                   # Feedback
    lambda x: np.mean(x),          # Aggregate
]

# ======================================================
# METRIC INVERSES
# ======================================================
METRIC_INVERSES = [
    lambda y: -np.log(y),          # GenerateDemand
    lambda y: -np.log(y),          # NavigateRoute
    lambda y: (y - 0.1) / 0.2,     # TrafficAssessment
    lambda y: y,                   # LoadCheck
    lambda y: y,                   # PackageHandling
    lambda y: -np.log(y),          # DeliveryExecution
    lambda y: -np.log(y),          # PaymentConfirmation
    lambda y: y,                   # Feedback
    lambda y: y,                   # Aggregate
]

# ======================================================
# METRIC → FEATURE MAP
# ======================================================
metric_feature_map = {
    "GenerateDemand": ['demand_volume', 'urgency'],
    "NavigateRoute": ['distance_km', 'planned_speed'],
    "TrafficAssessment": ['congestion', 'weather_risk'],
    "LoadCheck": ['capacity_ratio', 'current_load'],
    "PackageHandling": ['handling_time_min', 'fragility'],
    "DeliveryExecution": ['actual_speed', 'delay_minutes'],
    "PaymentConfirmation": ['payment_success', 'processing_time_sec'],
    "Feedback": ['satisfaction', 'complaint'],
    "Aggregate": 'ALL'  # special case: aggregate over all node metrics
}



class BranchBoundOptimizer:
    """
    Exact, non-recursive Branch & Bound optimizer for 1D continuous problems.
    """
    def __init__(
        self,
        tol=1e-3,
        max_depth=20,
        minimize=True,
        value_range=(0.0, 5.0)
    ):
        self.tol = tol
        self.max_depth = max_depth
        self.minimize = minimize
        self.value_range = value_range

    def optimize(self, features, y=None, metric_mask=None):
        y = np.zeros(3) if y is None else np.array(y[:3])
        base = np.mean(list(features.values())) + np.mean(y)

        # Initial interval (shifted by base)
        a0, b0 = self.value_range
        a0 += base
        b0 += base

        # Work list: (a, b, depth)
        work = [(a0, b0, 0)]

        best_x = None
        best_score = np.inf if self.minimize else -np.inf

        def better(s1, s2):
            return s1 < s2 if self.minimize else s1 > s2

        while work:
            a, b, depth = work.pop()

            mid = 0.5 * (a + b)

            # Evaluate midpoint
            mv = [f(mid) if m else 0.0 for f, m in zip(METRIC_FORMULAS, metric_mask)]
            score = sum(mv)

            if best_x is None or better(score, best_score):
                best_x = mid
                best_score = score

            # Termination condition
            if depth >= self.max_depth or (b - a) < self.tol:
                continue

            # Branch (NO pruning — exact)
            work.append((a, mid, depth + 1))
            work.append((mid, b, depth + 1))

        return best_x


class MetricsEvaluator:
    """Compute node metrics using feature subsets and exact B&B optimizer per metric."""
    def __init__(
        self,
        data_matrix,
        metric_formulas=METRIC_FORMULAS,
        metric_feature_map=metric_feature_map,
        feature_keys=FEATURE_KEYS,
        feature_target=None,
        metric_target=None,
        tol=1e-3,
        max_depth=20,
        minimize=False,
        value_range=(0.0, 5.0)
    ):
        self.data_matrix = data_matrix
        self.metric_formulas = metric_formulas
        self.metric_feature_map = metric_feature_map
        self.feature_keys = feature_keys
        self.feature_target = feature_target or [[1]*len(feature_keys) for _ in range(data_matrix.shape[0])]
        self.metric_target = metric_target or [[1]*len(metric_formulas) for _ in range(data_matrix.shape[0])]
        self.num_nodes = data_matrix.shape[0]

        # Create a B&B optimizer instance
        self.optimizer = BranchBoundOptimizer(
            tol=tol,
            max_depth=max_depth,
            minimize=minimize,
            value_range=value_range
        )

    def extract_features(self, node_idx):
        """Return active features for a node as {feature_key: value}."""
        row = self.data_matrix[node_idx]
        mask = self.feature_target[node_idx]
        features = {k: v for k, v, m in zip(self.feature_keys, row, mask) if m}
        return features

    def compute_node_metrics(self, node_idx, y=None):
        """
        Compute metrics for a single node using:
        - relevant features from metric_feature_map
        - B&B optimizer per metric
        """
        features = self.extract_features(node_idx)
        metric_mask = self.metric_target[node_idx]

        metric_values = {}

        for key, formula, mask in zip(METRIC_KEYS, self.metric_formulas, metric_mask):
            if mask:
                # Pick only the relevant features for this metric
                relevant_features = [features[f] for f in self.metric_feature_map[key] if f in features]
                x = np.mean(relevant_features) if relevant_features else 0.0

                # Optimize the scalar using B&B
                opt_value = self.optimizer.optimize(features={key: x}, y=y, metric_mask=[1])
                metric_values[key] = formula(opt_value)
            else:
                metric_values[key] = 0.0

        metric_values['score'] = sum(metric_values.values())
        return metric_values


In [ ]:
#METRIC_TARGET=[[1, 1, 1, 1, 1,1], [1, 1, 1, 1, 1,1],[1, 1, 1, 1, 1,1], [1, 1, 1, 1, 1,1],[1, 1, 1, 1, 1,1],[1, 1, 1, 1, 1,1], [1, 1, 1, 1, 0]]
import numpy as np

np.random.seed()

# Number of synthetic samples per node
n_samples = 3

# Number of nodes (including Aggregate)
nodes = list(GENERATOR_MAP.keys()) + ["Aggregate"]

# Candidate dimensions per node (example)
candidate_dims = [2, 2, 2, 2, 2, 2, 2, 2, 3]  # last one for Aggregate

def generate_synthetic_targets_per_node(generator_map, nodes, candidate_dims, n_samples=3):
    synthetic_targets = []

    for node_idx, node in enumerate(nodes):
        dim = candidate_dims[node_idx]
        for sample_id in range(n_samples):
            if node != "Aggregate":
                features = list(generator_map[node]().values())
                # Pad or truncate to match dim
                if len(features) >= dim:
                    features = features[:dim]
                else:
                    features = np.pad(features, (0, dim - len(features)), constant_values=0.5)
            else:
                # Aggregate node: just a vector of 0.5s
                features = np.full(dim, 0.5)

            synthetic_targets.append({
                "node": node,
                "sample_id": sample_id + 1,
                "target": np.array(features)
            })

    return synthetic_targets

# Generate targets
synthetic_targets = generate_synthetic_targets_per_node(GENERATOR_MAP, nodes, candidate_dims, n_samples)

# Check results
for t in synthetic_targets:
    print(f"Node {t['node']} Sample {t['sample_id']} Target: {t['target']}")

import numpy as np
import pandas as pd

np.random.seed(42)

# ---------------- GENERATE RAW DATA MATRIX ----------------
# For example, 100 samples
num_samples = 100
feature_list = []

# Collect features from each generator
for tp, gen in GENERATOR_MAP.items():
    samples = [gen() for _ in range(num_samples)]
    df = pd.DataFrame(samples)
    feature_list.append(df)

# Concatenate all columns horizontally
DATA_MATRIX = pd.concat(feature_list, axis=1).to_numpy()

# Normalize to 0-1
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0) + 1e-8)

# ---------------- SYNTHETIC TARGET GENERATOR ----------------
def generate_synthetic_targets_per_node(DATA_MATRIX, candidate_dims):
    """
    Generate per-node synthetic targets from DATA_MATRIX and candidate_dims.
    """
    num_nodes = len(candidate_dims)
    num_rows = DATA_MATRIX.shape[0]

    # Split features roughly equally per node
    feature_blocks = np.array_split(np.arange(DATA_MATRIX.shape[1]), num_nodes)

    targets = []
    for node_idx in range(num_nodes):
        row = DATA_MATRIX[node_idx % num_rows]
        features = row[feature_blocks[node_idx]]
        dim = candidate_dims[node_idx]

        # Pad or truncate to match candidate_dims
        if len(features) >= dim:
            sampled = features[:dim]
        else:
            sampled = np.pad(features, (0, dim - len(features)), constant_values=0.5)

        targets.append({'node_id': node_idx, 'target': sampled})

    return targets

# ---------------- EXAMPLE USAGE ----------------
D_graph = 8
candidate_dims = [2, 2, 2, 2, 2, 2, 2, 2, ]  # 9 nodes
synthetic_targets = generate_synthetic_targets_per_node(DATA_MATRIX, candidate_dims)

# ---------------- CHECK OUTPUT ----------------
for t in synthetic_targets:
    print(f"Node {t['node_id']} target size: {len(t['target'])}, values: {t['target']}")



In [ ]:
synthetic_targets

# 4. Fuzzy Hierarchical Multiplex Model

### A fuzzy hierarchical multiplex that is optimized to given data targets for given outer objectives according to some metrics. It then provides diagnostic plots and prints the formula for each node metric formula. Here is a pseudo coded method.

    -input
        -data targets
        -candidate dimension
        -data learning rates
        -metric mask
    -processes
        -Solves run inner optimal metrics
        -uses mask to discenr metrics
        -Evaluates metrics
        -trains an SVM per node according to dimensions
        -returns weights to outer layer
        -solves for the objective of outer layer
        -calculates outer contribution of metrics
        -calculates overal outer metrics
    -outputs
        -solves optimizaiotn of data problem
        -solves outer objective problem
        -produces an SVM per node for metric reversal to datapoint
        -plots diagnostics


## Creating the DSM based on Behavioural random walks.

### In this instance instead of assuming a DSM the metrics were taken as they were, couplings were implemented as in Bai paper, and a clever program was devised. The idea was simple, lets create a desing based on metrics that adheres to behavioural logic, which can be implemented using random walks. __data-->metrics-->RW-->DSM__... And so, like this we can optimize the desing based on metrics and anything that one can imagine

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
# Target Mask (which metrics apply to which node)
candidate_dims = [[2],[2],[2],[2],[2],[2],[2],[2],]#[1],]

def top_k_masked_probs(weights, k):
    """
    Keep only top-k weights, zero out the rest, renormalize.
    """
    if k >= len(weights):
        return weights / (weights.sum() + 1e-12)

    idx = np.argpartition(weights, -k)[-k:]
    mask = np.zeros_like(weights)
    mask[idx] = weights[idx]

    s = mask.sum()
    if s > 0:
        mask /= s
    return mask
def get_all_paths(G, C, node_types, start=0, end=None):
    """
    Return all simple paths that respect coupling constraints.
    """
    if end is None:
        end = G.shape[0] - 1

    D = G.shape[0]
    paths = []

    def coupling_ok(i, j):
        label = C[i, j]

        # Default direct coupling
        if "->" in label and "->C" not in label:
            src, dst = label.split("->")
            return src == node_types[i] and dst == node_types[j]

        # Escalation case (B->B->C)
        if label == "B->B->C":
            return node_types[i] == "B" and node_types[j] == "B"

        return False

    def dfs(node, path, visited):
        if node == end:
            paths.append(path.copy())
            return

        for nxt in range(D):
            if G[node, nxt] > 0 and nxt not in visited:
                if not coupling_ok(node, nxt):
                    continue

                visited.add(nxt)
                dfs(nxt, path + [nxt], visited)
                visited.remove(nxt)

    dfs(start, [start], {start})
    return paths



# =============================================================================
# EXTERNAL DEPENDENCIES & CONFIGURATION
# =============================================================================
# These variables are referenced in the original code but not defined.
# Assumed to be present in the execution environment.
# -----------------------------------------------------------------------------
# D_graph = ...
# DATA_MATRIX = ...
# METRIC_KEYS = [...]
# METRIC_TARGET = [...]
# METRIC_FORMULAS = [...]
# METRIC_INVERSES = {...}
# synthetic_targets = ...
# MetricsEvaluator = ... (Class)
# -----------------------------------------------------------------------------
# =============================================================================
# NEW: DETERMINISTIC DFS & PATH EVALUATOR
# =============================================================================


def evaluate_fixed_paths(paths, node_metrics, beta=0.3):
    """
    Replaces collect_and_select_best_walks.
    Scores the specific paths found by DFS.
    """
    walks = []
    best = None

    for path in paths:
        cost = 0.0
        quality = 0.0

        # Calculate Path Metrics
        for node_idx in path:
            m = node_metrics[node_idx]
            cost += m.get("cost", 0.0)
            quality += m.get("quality", 0.0)

        score = quality - 1.0 * cost # using default lambda_cost=1.0

        # DFS paths are unique, so count is always 1
        count = 1
        adjusted_score = score / (1 + beta * count)

        record = {
            "path": path,
            "score": score,
            "adjusted_score": adjusted_score,
            "cost": cost,
            "quality": quality,
            "count": count
        }
        walks.append(record)

        if best is None or adjusted_score > best["adjusted_score"]:
            best = record

    return walks, best

def dsm_walk(G, node_metrics, start=0, max_steps=50, lambda_cost=1.0):
    D = G.shape[0]
    target_node = D - 1  # Define Nlast

    # Force start at 0 if not provided
    current = start
    path = [current]
    visited = {current}

    cost = 0.0
    quality = 0.0

    # Metrics for the start node
    m_start = node_metrics[current]
    cost += m_start.get("cost", 0.0)
    quality += m_start.get("quality", 0.0)

    reached_target = False

    for _ in range(max_steps):
        # If we reached the last node, stop successfully
        if current == target_node:
            reached_target = True
            break

        weights = np.zeros(D)

        for j in range(D):
            # Allow visiting the target even if visited (though unlikely to revisit in DAG)
            # But generally prevent cycles
            if j == 0:
                continue
            if j in visited:
                continue

            q = node_metrics[j].get("quality", 0.0)
            c = node_metrics[j].get("cost", 0.0)

            # Standard probability weight
            weights[j] = G[current, j] * (q / (1 + c))

        if weights.sum() == 0:
            break

        weights /= weights.sum()
        nxt = np.random.choice(D, p=weights)

        m = node_metrics[nxt]
        cost += m.get("cost", 0.0)
        quality += m.get("quality", 0.0)

        path.append(nxt)
        visited.add(nxt)
        current = nxt

    # Recalculate score based on success
    score = quality - lambda_cost * cost

    # Heavy penalty if the walk did not reach Nlast
    if not reached_target:
        score = -1e9

    return path, score, cost, quality

from collections import defaultdict

def collect_and_select_best_walks(G, node_metrics, n_walks=300, beta=0.3):
    walks = []
    best = None
    path_counts = defaultdict(int)
    D = G.shape[0]

    for _ in range(n_walks):
        # Force start at 0
        start = 0
        path, score, cost, quality = dsm_walk(
            G, node_metrics, start
        )

        # Only record if it successfully reached the last node
        # (Score is -1e9 if it failed, per updated dsm_walk)
        if path[-1] != (D - 1):
            continue

        key = tuple(path)
        path_counts[key] += 1

        # novelty penalty
        adjusted_score = score / (1 + beta * path_counts[key])

        record = {
            "path": path,
            "score": score,
            "adjusted_score": adjusted_score,
            "cost": cost,
            "quality": quality,
            "count": path_counts[key]
        }
        walks.append(record)

        if best is None or adjusted_score > best["adjusted_score"]:
            best = record

    return walks, best

class CouplingState:
    """
    One-step coupling automaton.
    C = transient escalation after B->B->C
    """

    def __init__(self):
        self.state = "NORMAL"

    def update(self, coupling):
        if coupling == "B->B->C":
            self.state = "C"
        else:
            self.state = "NORMAL"

    def allows(self, next_node_type):
        """
        Rule:
        C → random, but NOT B
        """
        if self.state == "C" and next_node_type == "B":
            return False
        return True

def infer_node_types(node_metrics):
    """
    A = quality-dominant
    B = cost-dominant
    """
    node_types = []
    for m in node_metrics:
        q = m.get("quality", 0.0)
        c = m.get("cost", 0.0)
        node_types.append("A" if q >= c else "B")
    return node_types

def build_coupling_matrix(node_types):
    """
    Builds a D x D coupling-label matrix.
    C exists only as a coupling escalation (B->B->C).
    """
    D = len(node_types)
    C = np.empty((D, D), dtype=object)

    for i in range(D):
        for j in range(D):
            src = node_types[i]
            dst = node_types[j]

            if src == "B" and dst == "B":
                C[i, j] = "B->B->C"
            else:
                C[i, j] = f"{src}->{dst}"

    return C


def coupling_weight(coupling, metrics):
    cost = metrics.get("cost", 0.0)
    quality = metrics.get("quality", 0.0)

    if coupling == "A->A":          # Quality → Quality
        return quality

    if coupling == "A->B":          # Quality → Cost
        return quality

    if coupling == "B->A":          # Cost → Quality
        return quality

    if coupling == "B->B->C":       # Cost escalation signal
        return cost

    return 0.0

# Configuration
#candidate_dims = [[2], [2], [2], [2], [2], [2], [2], [2], [1]]
outer_generations = 1
inner_learning = 0.1
gamma_interlayer = 1
top_k = 21

# Initialize random state
np.random.seed()
seed = None  # Placeholder as per original logic

# Placeholder for Data Matrix generation (from original snippet)
# new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])


# =============================================================================
# HELPER CLASSES
# =============================================================================
class DSM_Tracker:
    """
    Tracks a DSM (Design Structure Matrix) layer and its residual.
    """
    def __init__(self, multiplex_layer):
        self.layer = multiplex_layer
        self.primary_dsm = None
        self.residual_dsm = None
        self.update_dsms()

    def update_dsms(self):
        if self.primary_dsm is None:
            # First time: store current DSM as reference
            self.primary_dsm = self.layer.chosen_Gmat.copy()
            self.residual_dsm = np.zeros_like(self.primary_dsm)
        else:
            # Update residual: current DSM minus primary
            current = self.layer.chosen_Gmat
            self.residual_dsm = current - self.primary_dsm

    def get_matrices(self):
        return self.primary_dsm, self.residual_dsm

    def print_matrices(self):
        print("\n--- Primary DSM ---")
        print(self.primary_dsm)
        print("\n--- Residual DSM ---")
        print(self.residual_dsm)


class DSM_Layer_Decomposer:
    """
    Manages the additive decomposition of DSM layers.
    """
    def __init__(self, baseline_matrix, mode='additive'):
        self.baseline_matrix = baseline_matrix.copy()
        self.current_total = baseline_matrix.copy()
        self.mode = mode
        self.layers = []
        self.residuals = []

    def add_snapshot(self, new_total_matrix):
        """
        Calculates the DELTA (change) between the new state and the previous state,
        stores that delta as a layer.
        """
        delta = new_total_matrix - self.current_total
        self.layers.append(delta.copy())

        # Update current tracker
        self.current_total = new_total_matrix.copy()

        # Calculate residual (Difference from the baseline)
        residual = self.current_total - self.baseline_matrix
        self.residuals.append(residual)

        layer_id = len(self.layers) - 1
        print(f"\n=== DSM LAYER {layer_id} CAPTURED ===")
        print(f"Layer Contribution (Delta):\n{np.round(delta, 3)}")

        return delta

    def get_reconstruction(self):
        return np.sum(self.layers, axis=0)


class SVM:
    """
    Metric-inverse multi-output SVM with epsilon-insensitive loss.
    """
    def __init__(self, input_dim, output_dim=None, metric_keys=None, lr=0.001, epsilon=0.1):
        self.input_dim = input_dim
        self.metric_keys = metric_keys

        if metric_keys is not None:
            self.output_dim = len(metric_keys)
        elif output_dim is not None:
            self.output_dim = output_dim
        else:
            self.output_dim = 1

        self.lr = lr
        self.epsilon = epsilon

        # Lazy initialization
        self.X_train = None
        self.y_train = None
        self.alpha = None
        self.b = None

    def train_step(self, X, y_true):
        X = np.array(X)
        y_true = np.array(y_true)
        n_samples = X.shape[0]

        if self.X_train is None:
            self.X_train = X.copy()
            self.y_train = y_true.copy()
            self.alpha = np.zeros((X.shape[1], self.output_dim))
            self.b = np.zeros(self.output_dim)

        # Linear kernel
        y_pred = X.dot(self.alpha) + self.b

        # Epsilon-insensitive loss
        diff = y_pred - y_true
        mask = np.abs(diff) > self.epsilon
        diff *= mask

        grad_alpha = X.T.dot(diff) / n_samples
        grad_b = diff.mean(axis=0)

        self.alpha -= self.lr * grad_alpha
        self.b -= self.lr * grad_b

        loss = np.mean(np.maximum(0, np.abs(y_pred - y_true) - self.epsilon))
        return loss

    def predict(self, X):
        X = np.array(X)
        y_pred = X.dot(self.alpha) + self.b

        if self.metric_keys is None:
            return y_pred

        # Apply metric inverses
        y_transformed = np.zeros_like(y_pred)
        for i, key in enumerate(self.metric_keys):
            inverse_fn = METRIC_INVERSES[key]
            # Handle potential list return from inverse_fn
            val_func = lambda y: inverse_fn(y)[0] if isinstance(inverse_fn(y), list) else inverse_fn(y)
            y_transformed[:, i] = np.array([val_func(y) for y in y_pred[:, i]])

        return y_transformed


class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        if seed is not None:
            np.random.seed(seed)

        self.D_graph = D_graph
        self.edge_threshold = edge_threshold
        self.gamma = gamma

        # Handle list vs int inputs for dimensions
        m_dim = max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim

        if inter_dim is not None:
            self.inter_dim = inter_dim[0] if isinstance(inter_dim, list) else inter_dim
        else:
            self.inter_dim = m_dim

        self.max_input = 2 * m_dim

        # Initialize weights
        self.weights = {}
        self.bias = {}
        for i in range(D_graph):
            for j in range(D_graph):
                if i != j:
                    w_init = np.random.uniform(-0.1, 0.1, (self.inter_dim, self.max_input))
                    self.weights[(i, j)] = w_init
                    self.bias[(i, j)] = np.zeros(self.inter_dim)

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        # Pad or truncation
        if len(concat) < self.max_input:
            concat = np.pad(concat, (0, self.max_input - len(concat)))
        else:
            concat = concat[:self.max_input]

        # Normalize
        concat = (concat - np.mean(concat)) / (np.std(concat) + 1e-12)

        # Activation
        v = self.weights[(i, j)].dot(concat) + self.bias[(i, j)]
        input_strength = np.clip(np.mean(np.abs(concat)), 0, 1)
        v = v * input_strength

        return 1 / (1 + np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i == j:
                    continue
                if abs(Gmat[i, j]) > self.edge_threshold:
                    acts[(i, j)] = self.compute_edge_activation(i, j, nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts) < 2:
            return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1, keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2, axis=1) / (A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds, stds) + 1e-12)
        np.fill_diagonal(corr, 0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat, nested_reps)
        if not acts:
            return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

def build_dsm_from_walks(D, paths):
    """
    Constructs a DSM where entry [i,j] is the probability
    that a successful process moves from i to j.
    """
    flow = np.zeros((D, D))

    # Count transitions
    for path in paths:
        for k in range(len(path) - 1):
            u, v = path[k], path[k+1]
            flow[u, v] += 1

    # Normalize by the total number of successful walks.
    # This prevents 'saturation'—if an edge is rarely used, it stays small.
    n_paths = len(paths)
    if n_paths > 0:
        flow = flow / n_paths

    np.fill_diagonal(flow, 0.0)
    return flow
# =============================================================================
# MAIN OPTIMIZER CLASS
# =============================================================================
class MetricDrivenRandomWalk:
    def __init__(self, coupling_matrix, node_metrics, max_steps=50):
        self.C = coupling_matrix
        self.node_metrics = node_metrics
        self.D = coupling_matrix.shape[0]
        self.max_steps = max_steps

    def transition_probs(self, i, k=2):
        w = np.zeros(self.D)

        for j in range(self.D):
            if i == j:
                continue

            weight = coupling_weight(self.C[i, j], self.node_metrics[j])

            # Directional heuristic (keep yours)
            if j > i:
                weight *= 1.2
            elif j < i:
                weight *= 0.8

            w[j] = weight

        # If nothing viable, fallback to uniform
        if w.sum() == 0:
            return np.ones(self.D) / self.D

        # Emphasize strong edges
        w = w ** 2

        # 🔒 HARD SPARSITY CONSTRAINT (TOP-K)
        w = top_k_masked_probs(w, k)

        return w


    def run(self, start=0):
        path = [start]
        current = start
        target_node = self.D - 1

        for _ in range(self.max_steps):
            if current == target_node:
                break

            probs = self.transition_probs(current)

            # Move
            current = np.random.choice(self.D, p=probs)
            path.append(current)

            if current == target_node:
                break

        return path

class Fuzzy_Hierarchical_Multiplex:
    def __init__(self, candidate_dims, D_graph,
                 synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True,
                 metrics=METRIC_KEYS, metric_mask=METRIC_TARGET):

        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag
        self.best_dim_per_node = [len(t)-1 for t in synthetic_targets]
        self.MM = metric_mask
        self.MK = metrics
        self.MKI = metrics + ['score']

        self.PLM = [[] for _ in range(self.D_graph)]
        self.PLMS = [[] for _ in range(self.D_graph)]
        self.nested_reps = [np.zeros(c[0]) for c in candidate_dims]

        # Inter-layer setup
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(0.0, 0.3, (D_graph, D_graph))
        np.fill_diagonal(self.chosen_Gmat, 0)

        self.l2_before, self.l2_after = [], []
        self.max_target_len = max(len(t['target']) for t in synthetic_targets)
        self.svm_lr = 0.01

        self.metric_traces = {k: [] for k in metrics}
        self.metric_traces_per_node = [{} for _ in range(self.D_graph)]

        # DSM optimization hyperparameters
        self.dsm_lr = 0.1
        self.dsm_l1 = 0.02
        self.dsm_clip = 1.0
        self.dsm_history = []
        self.dsm_cost_weight = 0.05

    def print_dsm_basic(self):
        D = self.D_graph
        print("\n=== DESIGN STRUCTURE MATRIX (DSM) : Gmat ===")
        header = "     " + " ".join([f"N{j:>4}" for j in range(D)])
        print(header)
        for i in range(D):
            row = "N{:>2} | ".format(i)
            for j in range(D):
                row += f"{self.chosen_Gmat[i, j]:>5.2f} "
            print(row)

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
                  steps=1000, lr_x=1, lr_y=0.010, lr_W=1,
                  decorrelate_metrics=False):

        # --- Initialize activations ---
        x = np.random.uniform(-0.6, 0.6, D_fcm)
        y = np.random.uniform(-0.1, 0.1, D_fcm)

        # L2 tracking
        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx][:len(target)] - target))

        # --- FCM updates ---
        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        for _ in range(steps):
            z = y.dot(W) + x
            Theta_grad_z = z - target
            Theta_grad_x = Theta_grad_z
            Theta_grad_y = Theta_grad_z.dot(W.T)
            Theta_grad_W = np.outer(y, Theta_grad_z)

            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)
            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        # --- Update nested representation ---
        self.nested_reps[node_idx][:len(x)] = x
        self.l2_after.append(np.linalg.norm(x - target))

        # --- Extract node features ---
        # Assuming MetricsEvaluator is a global or imported class
        metrics_evaluator = MetricsEvaluator(data_matrix=DATA_MATRIX)
        features = metrics_evaluator.extract_features(node_idx)
        feat_vals = np.array(list(features.values()))

        # --- Compute metrics scaled by activations + features ---
        metric_mask = METRIC_TARGET[node_idx]
        metric_values = {}

        for key, formula, mask in zip(METRIC_KEYS, METRIC_FORMULAS, metric_mask):
            if mask:
                weighted_input = np.mean(feat_vals)
                # Outer scale check
                outer_scale = getattr(self, 'best_node_weights', {}).get(node_idx, 1.0)
                if isinstance(outer_scale, (list, np.ndarray)):
                     # fallback if it was stored incorrectly in previous context
                    outer_scale = 1.0

                weighted_input *= outer_scale
                metric_val = formula(weighted_input)
                metric_values[key] = metric_val

                # STORE DATAPOINT
                self.metric_traces[key].append((weighted_input, metric_val))
            else:
                metric_values[key] = 0.0

        # --- Total score ---
        metric_values['score'] = sum(metric_values.values())

        # --- Build SVM Training Data ---
        metric_output_vals = np.array(
            [v for k, v in metric_values.items() if k not in ['score', 'x', 'feat_vals']]
        )

        # Lazy init per-node SVM
        if not hasattr(self, "node_svms"):
            self.node_svms = {}

        if node_idx not in self.node_svms:
            self.node_svms[node_idx] = SVM(
                input_dim=len(self.MK),
                output_dim=self.candidate_dims[node_idx][0],
                lr=self.svm_lr
            )

        svm = self.node_svms[node_idx]

        # Build SVM Input/Output
        x_in_full = np.zeros(len(self.MK))
        x_in_full[:len(metric_output_vals)] = metric_output_vals
        x_in = x_in_full.reshape(1, -1)

        y_out_full = np.zeros(self.candidate_dims[node_idx][0])
        y_out_full[:len(x)] = x
        y_out = y_out_full.reshape(1, -1)

        # Train SVM
        _ = svm.train_step(x_in, y_out)

        # --- Store PLMS trace ---
        self.PLMS[node_idx].append((float(weighted_input), metric_output_vals))

        if len(self.PLMS[node_idx]) % 100 == 0:
            print(f"Node {node_idx}, samples learned:", len(self.PLMS[node_idx]))

        # --- Compute inter-layer MI ---
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score, metric_values

    # ---------- OUTER LOOP (Topology Optimization) ----------
    def run_outer(self, outer_cost_limit=1000, alpha=0.0, additive_rate=0.5):
        """
        Generates an additive DSM layer.
        CRITICAL: Enforces strict constraints regardless of layer state.

        Constraints:
        1. Start Node == 0
        2. End Node == D-1
        3. Hamiltonian (Length == D, visits all nodes exactly once)
        4. CouplingState (Respects A/B/C transition rules)
        """
        node_metrics_list = self.capped_node_metrics
        D = self.D_graph

        # =========================================================
        # 1. METRIC SCORING & TENSOR CALCULATIONS
        # =========================================================
        raw_scores = np.array([m['score'] for m in node_metrics_list])
        total_raw = raw_scores.sum()
        if total_raw > outer_cost_limit:
            scale_factor = outer_cost_limit / total_raw
            for metrics in node_metrics_list:
                for key in self.MKI:
                    metrics[key] *= scale_factor
            raw_scores *= scale_factor

        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=False)
        self.weighted_fmt = fuzzy_tensor.copy()

        # Calculate Contributions
        node_contributions = np.zeros(D)
        for i in range(D):
            own_score = raw_scores[i]
            fmt_contrib = fuzzy_tensor[i, :, :].sum() - fuzzy_tensor[i, i, :].sum()
            node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib
        self.node_score_contributions = node_contributions

        # Correlation Penalty
        interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)
        if interaction_tensor is not None:
            fmt_mean = fuzzy_tensor.mean(axis=2)
            inter_mean = interaction_tensor.mean(axis=2)
            corr_penalty = 0.0
            for i in range(D):
                fmt_vec = fmt_mean[i, :]
                inter_vec = inter_mean[i, :]
                if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0, 1])**2
            corr_penalty /= D
        else:
            corr_penalty = 0.0
        self.correlation_penalty = corr_penalty

        # =========================================================
        # 2. STRICT PATHFINDING LOGIC
        # =========================================================

        node_metrics = self.capped_node_metrics
        node_types = infer_node_types(node_metrics)

        # --- INTERNAL VALIDATOR FUNCTION ---
        def is_strictly_valid(path):
            # 1. Topology Constraints
            if path[0] != 0: return False
            if path[-1] != (D - 1): return False
            if len(path) != D: return False # Must be Hamiltonian
            if len(set(path)) != D: return False # Unique nodes only

            # 2. Coupling Constraints (Simulation)
            state_machine = CouplingState()

            for k in range(len(path) - 1):
                u, v = path[k], path[k+1]
                type_u = node_types[u]
                type_v = node_types[v]

                # Check if transition is allowed by current state
                if not state_machine.allows(type_v):
                    return False

                # Determine coupling label for update
                if type_u == "B" and type_v == "B":
                    coupling = "B->B->C"
                else:
                    coupling = f"{type_u}->{type_v}"

                state_machine.update(coupling)

            return True

        # --- A. SEARCH EXISTING TOPOLOGY ---
        C_matrix = build_coupling_matrix(node_types)

        # We start DFS specifically at 0
        raw_paths = get_all_paths(
            self.chosen_Gmat,
            C_matrix,
            node_types,
            start=0,
            end=D - 1
        )

        valid_paths = [p for p in raw_paths if is_strictly_valid(p)]

        # --- B. SYNTHESIS FALLBACK (If topology is sparse/invalid) ---
        # If no valid paths exist in the current weights, we MUST synthesize one
        # to ensure the layer does not break the logic.
        if len(valid_paths) == 0:
            print(f" [Constraint Enforcement] No natural paths found. Synthesizing valid Hamiltonian permutations...")

            middle_nodes = [n for n in range(D) if n != 0 and n != (D-1)]
            attempts = 0

            # Try random permutations until we find one that satisfies CouplingState
            while len(valid_paths) < 3 and attempts < 1000:
                np.random.shuffle(middle_nodes)
                candidate = [0] + middle_nodes + [D-1]

                if is_strictly_valid(candidate):
                    valid_paths.append(candidate)

                attempts += 1

            if len(valid_paths) == 0:
                # Should be extremely rare unless constraints are impossible (e.g. all B nodes)
                print("CRITICAL WARNING: Constraints are too tight. No valid path exists.")
                # We do not fallback to invalid paths. The layer will be empty (Identity).

        # =========================================================
        # 3. LAYER CONSTRUCTION
        # =========================================================

        # Score the Valid Paths
        def path_sum_score(path, G):
            return sum(G[path[i], path[i+1]] for i in range(len(path) - 1))

        scored_paths = [
            (path, path_sum_score(path, self.chosen_Gmat))
            for path in valid_paths
        ]

        # Sort and Select Top-K
        K_paths = 3
        scored_paths.sort(key=lambda x: x[1], reverse=True)
        top_paths = [p for p, _ in scored_paths[:K_paths]]

        # Build Matrix from Paths
        G_layer = build_dsm_from_walks(D, top_paths)

        # Sparsity: Top-1 outgoing edge per node for this layer
        K_sparse = 2
        G_layer_sparse = np.zeros_like(G_layer)
        for i in range(D - 1):
            row = G_layer[i].copy()
            row[i] = 0.0
            if row.sum() == 0: continue
            idx = np.argsort(row)[-K_sparse:]
            G_layer_sparse[i, idx] = row[idx]
            s = G_layer_sparse[i].sum()
            if s > 0: G_layer_sparse[i] /= s

        # =========================================================
        # 4. REVIEW & UPDATE
        # =========================================================
        print(f"\n--- REVIEWING STRICT LAYER ({len(valid_paths)} valid paths) ---")
        for idx, (p, s) in enumerate(scored_paths[:K_paths]):
            path_str = " -> ".join([f"N{node}" for node in p])
            # Double check validation just for log assurance
            status = "VALID" if is_strictly_valid(p) else "INVALID"
            print(f"Path {idx+1} [{status}]: {path_str} (Score: {s:.3f})")

        # Apply Additive Update
        self.dsm_history.append(G_layer_sparse.copy())
        self.chosen_Gmat = self.chosen_Gmat + (additive_rate * G_layer_sparse)

        # Normalize
        max_val = np.max(self.chosen_Gmat)
        if max_val > 1.0:
            self.chosen_Gmat /= max_val

        # Collect final statistics (for visualization)
        self.walks, self.best_walk = collect_and_select_best_walks(
            self.chosen_Gmat,
            self.capped_node_metrics
        )

        dsm_cost = np.sum(np.abs(self.chosen_Gmat))
        combined_score = node_contributions.sum() - corr_penalty - (self.dsm_cost_weight * dsm_cost)

        self.print_dsm_basic()

        if not hasattr(self, "_node_contributions_history"):
            self._node_contributions_history = []
        self._node_contributions_history.append(node_contributions.copy())

        return node_metrics_list, combined_score, node_contributions

    def run(self, outer_generations=1, num_dsm_layers=5):
        best_score = -np.inf

        # Fix: Initialize Decomposer with a zero matrix
        baseline = np.zeros_like(self.chosen_Gmat)
        dsm_decomposer = DSM_Layer_Decomposer(baseline, mode='compounded')
        dsm_decomposer.current_total = self.chosen_Gmat.copy()

        print(f"Starting Optimization: {num_dsm_layers} Layers x {outer_generations} Gens")

        for layer_idx in range(num_dsm_layers):
            print(f"\n>>> STARTING LAYER {layer_idx + 1} / {num_dsm_layers} <<<")

            for gen in range(outer_generations):
                # 1. Inner Loop (Simulation)
                node_metrics_list = []
                for node_idx in range(self.D_graph):
                    full_target = self.synthetic_targets[node_idx]['target']
                    D_fcm = self.candidate_dims[node_idx][0]
                    target = full_target[:D_fcm]

                    _, _, _, _, metrics = self.run_inner(node_idx, target, D_fcm)
                    node_metrics_list.append(metrics)

                self.capped_node_metrics = node_metrics_list

                # 2. Outer Loop (Topology Optimization)
                _, capped_score, _ = self.run_outer()
                best_score = max(best_score, capped_score)

                # --- PROGRESS PRINT ---
                print(f" [Layer {layer_idx+1}] Gen {gen+1}/{outer_generations} | Score: {capped_score:.4f}", end='\r')

            print("") # Newline after generation loop finishes

            # Snapshot the state
            dsm_decomposer.add_snapshot(self.chosen_Gmat)

        self.dsm_layers = dsm_decomposer.layers
        print("\nOptimization Complete.")
        return best_score

    # ---------- VISUALIZATIONS & ANALYSIS ----------

    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14, 3))
        for i in range(self.D_graph):
            dim_i = self.candidate_dims[i][0]
            base = self.nested_reps[i][:dim_i]
            reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base

            y_true = self.synthetic_targets[i]['target'][:len(y_sel)]
            if len(y_true) < len(y_sel):
                y_true = np.pad(y_true, (0, len(y_sel) - len(y_true)), "constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1, self.D_graph, i + 1)
            plt.fill_between(range(len(y_min)), y_min, y_max, color='skyblue', alpha=0.4, label='Elite Interval')
            plt.plot(y_sel, 'k-', lw=2, label='Estimated')
            plt.plot(y_true, 'r--', lw=2, label='True')
            plt.ylim(0, 1.05)
            plt.title(f"Node {i + 1}")
            if i == 0: plt.legend()
        plt.tight_layout()
        plt.show()

    def plot_nested_activations(self):
        plt.figure(figsize=(12, 3))
        for i, rep in enumerate(self.nested_reps):
            dim_i = self.candidate_dims[i][0]
            rep_i = rep[:dim_i]
            plt.subplot(1, self.D_graph, i + 1)
            plt.bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
            plt.ylim(0, 1)
            plt.title(f"Node {i + 1}")
        plt.tight_layout()
        plt.show()

    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i != j and abs(self.chosen_Gmat[i, j]) > 0.02:
                    G.add_edge(i, j, weight=self.chosen_Gmat[i, j])

        node_sizes = [self.best_dim_per_node[i] * 200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight'] > 0 else 'red' for _, _, d in G.edges(data=True)]
        edge_widths = [abs(d['weight']) * 3 for _, _, d in G.edges(data=True)]

        pos = nx.spring_layout(G)
        plt.figure(figsize=(6, 6))
        nx.draw(G, pos, node_size=node_sizes, node_color='skyblue',
                edge_color=edge_colors, width=edge_widths, arrows=True, with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()

    def print_interactions(self, return_tensor=True, verbose=True):
        D_graph = self.D_graph
        inter_dim = self.inter_layer.inter_dim
        inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

        acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
        if not acts:
            if verbose:
                print("No active edges above threshold.")
            return inter_tensor if return_tensor else None

        for (i, j), vec in acts.items():
            inter_tensor[i, j, :] = vec
            if verbose:
                act_str = ", ".join([f"{v:.3f}" for v in vec])
                print(f"Node {i} -> Node {j}: [{act_str}]")
        return inter_tensor if return_tensor else None

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
        metrics_keys = self.MK
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor = np.zeros((D, D, num_metrics))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        node_metrics = []
        for i, rep in enumerate(self.nested_reps):
            metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
            node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
        node_metrics = np.array(node_metrics)

        for i in range(D):
            for j in range(D):
                if i == j:
                    tensor[i, j, :] = node_metrics[j]
                else:
                    weight = np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                    tensor[i, j, :] = weight * node_metrics[j]

        if normalize:
            tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

        if verbose:
            print("Fuzzy Metric Tensor shape:", tensor.shape)

        return tensor

    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=None):
        if metrics_keys is None:
            metrics_keys = self.MK
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4 * num_metrics, 4))
        if num_metrics == 1: axes = [axes]

        im = None
        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:, :, k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j, i, f"{data[i, j]:.2f}", ha='center', va='center', color='white', fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        metrics_keys = self.MK
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D, D, num_metrics, 2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)

            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx, :] = [m[k] for k in metrics_keys]

            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            for j in range(D):
                tensor_bounds[i, j, :, 0] = lower_i
                tensor_bounds[i, j, :, 1] = upper_i

        return tensor_bounds

    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = self.MK
        M_actual = len(metrics_keys)

        mean_vals = (fmt_tensor_bounds[:, :, :, 0] + fmt_tensor_bounds[:, :, :, 1]) / 2
        mean_vals = mean_vals.mean(axis=1)  # mean across targets
        mean_vals = mean_vals.mean(axis=0, keepdims=True)  # mean across nodes

        if hasattr(self, 'best_alpha') and hasattr(self, 'best_w_contrib'):
            mean_weight = (self.best_alpha * self.best_w_contrib).mean()
            mean_vals = mean_vals * mean_weight

        fig, ax = plt.subplots(figsize=(1.2 * M_actual + 4, 2))
        im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

        vmin, vmax = mean_vals.min(), mean_vals.max()
        for i in range(mean_vals.shape[0]):
            for k in range(M_actual):
                val = mean_vals[i, k]
                color = 'white' if val < (vmin + 0.5 * (vmax - vmin)) else 'black'
                ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

        ax.set_xticks(range(M_actual))
        ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
        ax.set_yticks([0])
        ax.set_yticklabels(['Mean across nodes'])
        ax.set_title("Weighted FMT with Bounds")
        fig.colorbar(im, ax=ax, label='Weighted Mean Metric Value')
        plt.tight_layout()
        plt.show()

    def plot_node_score_contribution(self, metrics_keys=None):
        if metrics_keys is None:
            metrics_keys = self.MK
        D = self.D_graph
        node_contributions = np.array(self.node_score_contributions)

        if hasattr(self, 'weighted_fmt'):
            fuzzy_tensor = np.array(self.weighted_fmt)
        else:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        fuzzy_tensor_norm = (fuzzy_tensor - fuzzy_tensor.min()) / (fuzzy_tensor.max() - fuzzy_tensor.min() + 1e-12)
        fmt_matrix = fuzzy_tensor_norm.sum(axis=2)
        np.fill_diagonal(fmt_matrix, 0)

        raw_matrix = np.zeros((D, D))
        np.fill_diagonal(raw_matrix, node_contributions)

        total_matrix = raw_matrix + fmt_matrix

        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "Normalized FMT Contribution", "Total Contribution"]

        im = None
        for ax, mat, title in zip(axes, matrices, titles):
            im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    ax.text(j, i, f"{mat[i, j]:.2f}", ha='center', va='center', color='white', fontsize=8)
            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i + 1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i + 1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Contribution Value')
        plt.tight_layout()
        plt.show()

    def plot_fmt_with_run_metrics(self, metrics_keys=None):
        if metrics_keys is None:
            metrics_keys = self.MK
        D = self.D_graph
        M_actual = len(metrics_keys)

        if not hasattr(self, 'capped_node_metrics'):
            raise ValueError("No node metrics available. Run run_outer() first.")

        weighted_fmt = np.zeros((D, D, M_actual))
        for i in range(D):
            for j in range(D):
                for k, key in enumerate(metrics_keys):
                    val = self.capped_node_metrics[j][key]
                    if i != j:
                        val *= np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                    weighted_fmt[i, j, k] = val

        for i in range(D):
            for k in range(M_actual):
                if not METRIC_TARGET[i][k]:
                    weighted_fmt[i, :, k] = 0.0

        mean_vals = weighted_fmt.mean(axis=1)

        fig, ax = plt.subplots(figsize=(1.2 * M_actual + 4, 0.35 * D + 4))
        im = ax.imshow(mean_vals, cmap='viridis', aspect='auto')

        vmin, vmax = mean_vals.min(), mean_vals.max()
        for i in range(D):
            for k in range(M_actual):
                val = mean_vals[i, k]
                color = 'white' if val < (vmin + 0.5 * (vmax - vmin)) else 'black'
                ax.text(k, i, f"{val:.2f}", ha='center', va='center', color=color, fontsize=8)

        ax.set_xticks(range(M_actual))
        ax.set_xticklabels(metrics_keys[:M_actual], rotation=45, ha='right')
        ax.set_yticks(range(D))
        ax.set_yticklabels([f"Node {i}" for i in range(D)])
        ax.set_title("Weighted FMT Metrics (Actual Run Output)")
        fig.colorbar(im, ax=ax, label='Weighted Metric Value')
        plt.tight_layout()
        plt.show()

    def collect_fmt_datapoints(self):
        self.fmt_datapoints = {k: [] for k in self.MK}
        for node_idx in range(self.D_graph):
            if len(self.PLMS[node_idx]) == 0:
                continue
            for weighted_input_val, metric_vals in self.PLMS[node_idx]:
                for m, key in enumerate(self.MK):
                    if m < len(metric_vals):
                        self.fmt_datapoints[key].append((weighted_input_val, metric_vals[m]))

    def collect_metric_traces_per_node(self):
        self.metric_traces_per_node = [{} for _ in range(self.D_graph)]
        for node_idx in range(self.D_graph):
            self.metric_traces_per_node[node_idx] = {k: [] for k in self.MK}
            if len(self.PLMS[node_idx]) == 0:
                continue
            for weighted_input_val, metric_vals in self.PLMS[node_idx]:
                for m, key in enumerate(self.MK):
                    if m < len(metric_vals):
                        self.metric_traces_per_node[node_idx][key].append((weighted_input_val, metric_vals[m]))

    def plot_fmt_per_datapoint(self, top_k=21, span=0.3, grid_size=100):
        if not hasattr(self, 'fmt_datapoints'):
            self.collect_fmt_datapoints()

        for key, formula in zip(self.MK, METRIC_FORMULAS):
            if key not in self.fmt_datapoints or len(self.fmt_datapoints[key]) == 0:
                continue

            data = np.array(self.fmt_datapoints[key])
            x_data, y_data = data[:, 0], data[:, 1]

            x_curve = np.linspace(x_data.min() - span, x_data.max() + span, grid_size)
            y_curve = np.array([formula(x) for x in x_curve])

            plt.figure(figsize=(6, 4))
            plt.scatter(x_data, y_data, alpha=0.6, label="FMT datapoints")
            plt.plot(x_curve, y_curve, 'r', lw=2, label="Metric equation")
            plt.xlabel("Weighted Input")
            plt.ylabel(f"{key} (FMT)")
            plt.title(f"FMT per datapoint - Metric: {key}")
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()

    def plot_metric_equations_per_node(self, grid_size=100, span=0.3):
        if not hasattr(self, 'metric_traces_per_node'):
            self.collect_metric_traces_per_node()

        for node_idx in range(self.D_graph):
            node_traces = self.metric_traces_per_node[node_idx]
            if all(len(v) == 0 for v in node_traces.values()):
                continue

            plt.figure(figsize=(6, 4))
            for key, formula in zip(self.MK, METRIC_FORMULAS):
                if key not in node_traces or len(node_traces[key]) == 0:
                    continue
                data = np.array(node_traces[key])
                x_data, y_data = data[:, 0], data[:, 1]

                x_curve = np.linspace(x_data.min() - span, x_data.max() + span, grid_size)
                y_curve = np.array([formula(x) for x in x_curve])

                plt.scatter(x_data, y_data, alpha=0.6, label=f"{key} datapoints")
                plt.plot(x_curve, y_curve, 'r', lw=2, label=f"{key} equation")

            plt.xlabel("Weighted Input")
            plt.ylabel("Metric Value")
            plt.title(f"Node {node_idx} - Metric Equations")
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()


# =============================================================================
# MAIN EXECUTION BLOCK
# =============================================================================

if __name__ == "__main__":
    # Ensure necessary globals exist before running; otherwise this block is illustrative
    try:
        optimizer = Fuzzy_Hierarchical_Multiplex(
            candidate_dims, D_graph,
            synthetic_targets,
            inner_learning, gamma_interlayer=0,
            causal_flag=False
        )

        # Run Optimization
        metrics_list = optimizer.run()

        # Visualizations
        optimizer.plot_pointwise_minmax_elite()
        optimizer.plot_nested_activations()

        # Compute FMT with elite bounds
        #fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k + 10)

        # Plot as heatmaps
        #optimizer.plot_fmt_with_run_metrics()

        # Compute fuzzy multiplex tensor
        #fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=False)
        #optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

        # Plot Contributions & Graph
        optimizer.plot_node_score_contribution()
        optimizer.plot_outer_fuzzy_graph()

        # Interactions
       # tensor = optimizer.print_interactions()
        #print("Tensor shape:", tensor.shape, '\n', tensor)

        # Datapoints & Equations
        optimizer.collect_fmt_datapoints()
        optimizer.plot_fmt_per_datapoint()
        optimizer.collect_metric_traces_per_node()
        optimizer.plot_metric_equations_per_node()

        # DSM Tracking Demo
        dsm_tracker = DSM_Tracker(optimizer)

        # Run extra DSM update
        optimizer.run_outer()
        dsm_tracker.update_dsms()

        # Retrieve matrices
        primary, residual = dsm_tracker.get_matrices()
        print("Primary DSM:\n", primary)
        print("Residual DSM:\n", residual)

    except NameError as e:
        print(f"Error: Missing external dependency definition. \n{e}")
        print("Please ensure D_graph, DATA_MATRIX, METRIC_KEYS, etc. are defined.")

# 5. Comparison of dummy, initial DSM and optimized DSM

In [ ]:
def compare_dsm_three_way(optimizer,
                          baseline_dsm,
                          outer_steps=10,
                          threshold=0.05):
    """
    Computes baseline DSM comparison with initial and optimized DSMs.
    Automatically aligns matrices by size to avoid broadcasting errors.

    Parameters:
        optimizer: object with .chosen_Gmat and .run_outer()
        baseline_dsm: np.array or pd.DataFrame (baseline DSM)
        outer_steps: int, number of optimization iterations
        threshold: float, edge threshold for graph drawing
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import networkx as nx

    # Convert to numpy if DataFrame
    if isinstance(baseline_dsm, pd.DataFrame):
        baseline_dsm = baseline_dsm.values.copy()

    D = baseline_dsm.shape[0]

    # --------------------------------------------------
    # 1. INITIAL MODEL DSM
    # --------------------------------------------------
    initial_dsm = optimizer.chosen_Gmat.copy()

    # Ensure shapes match
    if initial_dsm.shape != baseline_dsm.shape:
        raise ValueError(f"Shape mismatch: initial DSM {initial_dsm.shape} vs baseline DSM {baseline_dsm.shape}")

    # --------------------------------------------------
    # 2. RUN DSM OPTIMIZATION
    # --------------------------------------------------
    for _ in range(outer_steps):
        optimizer.run_outer()

    optimized_dsm = optimizer.chosen_Gmat.copy()

    # --------------------------------------------------
    # 3. DELTAS
    # --------------------------------------------------
    delta_init_vs_base = initial_dsm - baseline_dsm
    delta_opt_vs_base = optimized_dsm - baseline_dsm
    delta_opt_vs_init = optimized_dsm - initial_dsm

    # --------------------------------------------------
    # 4. NUMERICAL SUMMARY
    # --------------------------------------------------
    def summarize(label, delta):
        return {
            "label": label,
            "L1": np.sum(np.abs(delta)),
            "L2": np.linalg.norm(delta),
            "max": np.max(np.abs(delta))
        }

    summaries = [
        summarize("Initial − Baseline", delta_init_vs_base),
        summarize("Optimized − Baseline", delta_opt_vs_base),
        summarize("Optimized − Initial", delta_opt_vs_init),
    ]

    print("\n=== TRIPLE DSM COMPARISON ===")
    for s in summaries:
        print(f"{s['label']:<22} | "
              f"L1={s['L1']:.4f}  "
              f"L2={s['L2']:.4f}  "
              f"MaxΔ={s['max']:.4f}")

    # --------------------------------------------------
    # 5. HEATMAPS
    # --------------------------------------------------
    mats = [
        baseline_dsm,
        initial_dsm,
        optimized_dsm,
        delta_opt_vs_base
    ]
    titles = [
        "Baseline DSM",
        "Initial DSM",
        "Optimized DSM",
        "Optimized − Baseline"
    ]

    fig, axes = plt.subplots(1, 4, figsize=(20, 4))
    for ax, mat, title in zip(axes, mats, titles):
        im = ax.imshow(mat, cmap="coolwarm")
        ax.set_title(title)
        ax.set_xticks(range(D))
        ax.set_yticks(range(D))
    fig.colorbar(im, ax=axes, fraction=0.025, pad=0.04)
    plt.tight_layout()
    plt.show()

    # --------------------------------------------------
    # 6. GRAPH COMPARISON
    # --------------------------------------------------
    def draw_graph(Gmat, title):
        G = nx.DiGraph()
        for i in range(Gmat.shape[0]):
            for j in range(Gmat.shape[1]):
                if i != j and abs(Gmat[i, j]) > threshold:
                    G.add_edge(i, j, weight=Gmat[i, j])

        pos = nx.shell_layout(G)
        edge_colors = ["green" if d["weight"] > 0 else "red"
                       for _, _, d in G.edges(data=True)]
        widths = [abs(d["weight"]) * 3 for _, _, d in G.edges(data=True)]

        plt.figure(figsize=(7, 7))
        nx.draw(G, pos,
                node_color="lightblue",
                node_size=600,
                edge_color=edge_colors,
                width=widths,
                arrows=True,
                with_labels=True)
        plt.title(title)
        plt.show()

    draw_graph(baseline_dsm, "Baseline DSM")
    draw_graph(initial_dsm, "Initial DSM")
    draw_graph(optimized_dsm, "Optimized DSM")

    return {
        "baseline": baseline_dsm,
        "initial": initial_dsm,
        "optimized": optimized_dsm,
        "delta_init_vs_base": delta_init_vs_base,
        "delta_opt_vs_base": delta_opt_vs_base,
        "delta_opt_vs_init": delta_opt_vs_init,
        "summaries": summaries
    }


In [ ]:
results = compare_dsm_three_way(
    optimizer=optimizer,
    baseline_dsm=DSM_MATRIX,  # 8×8
    outer_steps=0
)


# 6. Evaluation of the results to a canonical basis

In [ ]:
# ==========================================
# ALL-IN-ONE DSM + Cost/Quality Evaluation
# ==========================================
import numpy as np
import matplotlib.pyplot as plt

# ---- Helper: Evaluate cost/quality ----
def evaluate_cost_quality_for_dsm(evaluator, dsm_matrix):
    """
    Apply DSM effect (e.g., as a weight) to data_matrix and compute cost & quality.
    Currently uses evaluator as-is. In a real scenario, DSM interactions could modulate features.
    """
    return evaluator.compute_system_metrics(df_post)  # using post-optimized execution data

# ---- Run DSM comparison & get matrices ----
dsm_results = compare_dsm_three_way(
    optimizer=optimizer,
    baseline_dsm=DSM_MATRIX,  # 8×8 BAI DSM
    outer_steps=1
)

baseline_dsm  = dsm_results["baseline"]
initial_dsm   = dsm_results["initial"]
optimized_dsm = dsm_results["optimized"]

# ---- Evaluate cost & quality for each DSM ----
baseline_cost, baseline_quality = evaluate_cost_quality_for_dsm(evaluator, baseline_dsm)
init_cost, init_quality         = evaluate_cost_quality_for_dsm(evaluator, initial_dsm)
opt_cost, opt_quality           = evaluate_cost_quality_for_dsm(evaluator, optimized_dsm)

# ---- Compute DSM deltas ----
def dsm_delta_metrics(A, B):
    delta = A - B
    return np.sum(np.abs(delta)), np.linalg.norm(delta), np.max(np.abs(delta))

deltas = {
    "Initial − Baseline"   : dsm_delta_metrics(initial_dsm, baseline_dsm),
    "Optimized − Baseline" : dsm_delta_metrics(optimized_dsm, baseline_dsm),
    "Optimized − Initial"  : dsm_delta_metrics(optimized_dsm, initial_dsm)
}

# ---- Print cost/quality comparison ----
print("\n=== COST / QUALITY COMPARISON ===")
print(f"Baseline   | Cost: {baseline_cost:.4f} | Quality: {baseline_quality:.4f}")
print(f"Initial    | Cost: {init_cost:.4f} | Quality: {init_quality:.4f}")
print(f"Optimized  | Cost: {opt_cost:.4f} | Quality: {opt_quality:.4f}")

# ---- Optional: Heatmaps ----
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, mat, title in zip(axes,
                          [baseline_dsm, initial_dsm, optimized_dsm],
                          ["Baseline DSM", "Initial DSM", "Optimized DSM"]):
    im = ax.imshow(mat, cmap="coolwarm", vmin=-1, vmax=1)
    ax.set_title(title)
plt.colorbar(im, ax=axes, fraction=0.03, pad=0.04)
plt.tight_layout()
plt.show()


In [ ]:

print("=== DUMMY RESULTS ===")
print(f"1. Structural Complexity Cost (DSM Coupling): {structural_cost:.4f}")
#print(f"   (High value = High integration effort between Route, Traffic, Load, etc.)")
print(f"2. Operational Cost Index (Avg per delivery):  {avg_op_cost:.4f}")
print(f"3. System Quality Score (Time & Satisfaction): {avg_quality:.4f}")


print("\n=== COST / QUALITY COMPARISON ===")
print(f"Dummy Opt  | Cost: {avg_op_cost:.4f} | Quality: {avg_quality:.4f}")
print(f"Baseline   | Cost: {baseline_cost:.4f} | Quality: {baseline_quality:.4f}")
print(f"Initial    | Cost: {init_cost:.4f} | Quality: {init_quality:.4f}")
print(f"Optimized  | Cost: {opt_cost:.4f} | Quality: {opt_quality:.4f}")



### The cost and the Quality are never intened for optimizaton, however they could be.

# 7. Pareto Front Generation